In [22]:
# DS LIBS
import numpy as np
import pandas as pd

# VISUALIZATION LIBS
import cv2
from pprint import pprint

# OS LIBS
import os
import config
from pathlib import Path


# DL LIBS
import unet
import torch
import torch.nn as nn
from torch.nn.functional import relu
from torchvision import models, transforms
from sklearn.model_selection import train_test_split



device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print("Using: " + str(device))

Using: cpu


In [8]:
def prep_image(file_path:str):
    
    # IMAGE TRANSFORMS
    img_array = cv2.imread(file_path)
    img_array = cv2.resize(img_array, (config.IMAGE_SIZE , config.IMAGE_SIZE))
    #img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    
    # GET IMAGE STATISTICS
    img_mean, img_std = np.mean(img_array, axis=(0, 1)), np.std(img_array, axis=(0, 1))

    # TORCH TRANSFORMS
    tensor_transforms = transforms.Compose([transforms.ToTensor(),
                                             transforms.Normalize(mean=img_mean,std=img_std)
                                            ])
    
    # APPLY TENSOR TRANSFORMS TO IMAGE 
    return tensor_transforms(img_array) 

In [9]:
# TESTING OUR IMAGES
test_path = 'D4-Control-6-1_1_MMStack_Pos0.ome.tif'
full_path = os.path.join(config.X_TRAIN_DATA, test_path)
test_img = cv2.imread(full_path)
print(test_img.shape)


test_path_2 = 'Particle_Thresholded_D4-Control-6-1_1_MMStack_Pos0.ome.png'
full_path_2 = os.path.join(config.Y_TRAIN_DATA, test_path_2)
test_img_2 = cv2.imread(full_path)
print(test_img_2.shape)

(2048, 2048, 3)
(2048, 2048, 3)


In [10]:
def prep_image_folder(folder_path:str) -> list:
    
    imgs:list = []
    
    for file_path in os.listdir(folder_path):
        full_path:str = os.path.join(folder_path, file_path)
    
        imgs.append(prep_image(full_path))
    
    return imgs

In [11]:
raw:list = []
gt:list = []


# PREP IMAGE FOLDERS
raw:list = prep_image_folder(config.X_TRAIN_DATA)
gt:list = prep_image_folder(config.Y_TRAIN_DATA)